# LSO Results Visualization

This notebook shows images along with their smile scores for latent and LoRA space optimization results.

The first part of this notebook implements general visualization functionalities, while the second (lengthy) part creates visualizations specifically for the experiment results.

## Image Grids

Visualize all available images of an optimization run.

### Optimized 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def grid_optimized(version, iters, idxs):
    """
    Plot a grid of optimized images.
    Args:
		version: string, version of the experiment
		iters: list of iteration numbers (rows)
		idxs:  list of sample indices (columns)
    """
    n_rows = len(iters)
    n_cols = len(idxs)
    fig, axes = plt.subplots(
        n_rows, n_cols,
        figsize=(4 * n_cols, 4 * n_rows),  # ~4" per subplot
        squeeze=False
    )
    fig.subplots_adjust(hspace=0.4)

    # Load scores
    scores = np.load(f"../results/{version}/results.npz", allow_pickle=True)

    for i, it in enumerate(iters):
        for j, idx in enumerate(idxs):
            a = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_opt/{idx}.png')
            ax_opt = axes[i][j]
            ax_opt.imshow(a)
            ax_opt.set_title(f"{scores['opt_point_properties'][it + idx].item():.2f}")
            ax_opt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
grid_optimized(
    version="ctrloralter_gbo_28",
    iters=list(range(0, 15, 5)),
    idxs=[0,1,2,3,4],
)

### Inital & Optimized 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def grid_initial_optimized(version, iters, idxs):
    """
    Plot a grid of initially sampled and optimized images.
    Args:
		version: string, version of the experiment
		iters: list of iteration numbers (rows)
		idxs:  list of sample indices (columns)
    """
    n_rows = len(iters)
    n_cols = len(idxs)
    fig, axes = plt.subplots(
        n_rows, n_cols * 2,
        figsize=(4 * n_cols * 2, 4 * n_rows),  # ~4" per subplot
        squeeze=False
    )
    fig.subplots_adjust(hspace=0.4)

    # Load scores
    scores = np.load(f"../results/{version}/results.npz", allow_pickle=True)

    for i, it in enumerate(iters):
        for j, idx in enumerate(idxs):
            # Left: initially sampled
            a = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_init/{idx}.png')
            ax_init = axes[i][2*j]
            ax_init.imshow(a)
            ax_init.set_title(f"Initial ({scores['init_point_properties'][it + idx].item():.2f})")
            ax_init.axis('off')

            # Right: optimized
            b = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_opt/{idx}.png')
            ax_opt = axes[i][2*j + 1]
            ax_opt.imshow(b)
            ax_opt.set_title(f"Optimized ({scores['opt_point_properties'][it + idx].item():.2f})")
            ax_opt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
grid_initial_optimized(
    version="sd_dngo_04",
    iters=list(range(0, 20, 5)),
    idxs=[0,1,2,3,4],
)

### Original & Initial & Optimized

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def grid_original_initial_optimized(version, iters, idxs):
    """
    Plot a grid of original, initially sampled, and optimized images.
    Args:
		version: string, version of the experiment
		iters: list of iteration numbers (rows)
		idxs:  list of sample indices (columns)
    """
    n_rows = len(iters)
    n_cols = len(idxs)
    fig, axes = plt.subplots(
        n_rows, n_cols * 3,
        figsize=(4 * n_cols * 3, 4 * n_rows),  # ~4" per subplot
        squeeze=False
    )
    fig.subplots_adjust(hspace=0.4)

    # Load scores
    scores = np.load(f"../results/{version}/results.npz", allow_pickle=True)

    for i, it in enumerate(iters):
        for j, idx in enumerate(idxs):
            # Left: original
            a = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_orig/{idx}.png')
            ax_orig = axes[i][3*j]
            ax_orig.imshow(a)
            ax_orig.set_title(f"Iteration {it}  Top {idx}\nOriginal ({scores['orig_point_properties'][it + idx].item():.2f})")
            ax_orig.axis('off')

            # Middle: initially sampled
            b = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_init/{idx}.png')
            ax_init = axes[i][3*j + 1]
            ax_init.imshow(b)
            ax_init.set_title(f"Initial ({scores['init_point_properties'][it + idx].item():.2f})")
            ax_init.axis('off')

            # Right: optimized
            c = Image.open(f'/BS/optdif/work/results/{version}/data/samples/iter_{it}/img_opt/{idx}.png')
            ax_opt = axes[i][3*j + 2]
            ax_opt.imshow(c)
            ax_opt.set_title(f"Optimized ({scores['opt_point_properties'][it + idx].item():.2f})")
            ax_opt.axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
grid_original_initial_optimized(
    version="ctrloralter_gbo_28",
    iters=list(range(0, 20, 5)),
    idxs=[0,1],
)

## Smile Scores Visualization

Plots available smile scores and statistics for each iteration.

### Optimized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def scores_optimized(version):

	# Load results from the npz file
	results = np.load(f"../results/{version}/results.npz", allow_pickle=True)

	# Convert the npz file to a pandas DataFrame
	df = pd.DataFrame({
		'opt': list(results['opt_point_properties']),
		'model_version': list(results['opt_model_version']),
	})

	# Group by model version and aggregate
	df = df.groupby('model_version').agg({
		'opt': ['mean', 'min', 'max'],
	}).reset_index()

	fig, ax = plt.subplots(figsize=(10, 6))
	ax.plot(df['model_version'], df['opt']['mean'], marker='o', label='Optimized Mean')
	ax.fill_between(df['model_version'], df['opt']['min'], df['opt']['max'], alpha=0.2, label='Optimized Min-Max Range')
	ax.set_xlabel('Iteration')
	ax.set_ylabel('Values')
	ax.set_title('Mean Objective Values of the Top Five Samples per Iteration')
	ax.set_ylim(0, 5)
	ax.axhline(y=2, color='gray', linestyle='--', label='Input Max')
	ax.legend()
	plt.xticks(ticks=range(len(df['model_version'])), labels=df['model_version'], rotation=45)

	plt.tight_layout()
	plt.show()

In [ ]:
scores_optimized(version="ctrloralter_gbo_28")

### Initial & Optimized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def scores_initial_optimized(version):

	# Load results from the npz file
	results = np.load(f"../results/{version}/results.npz", allow_pickle=True)

	# Convert the npz file to a pandas DataFrame
	df = pd.DataFrame({
		'opt': list(results['opt_point_properties']),
		'init': list(results['init_point_properties']),
		'model_version': list(results['opt_model_version']),
	})

	# Group by model version and aggregate
	df = df.groupby('model_version').agg({
		'opt': ['mean', 'min', 'max'],
		'init': ['mean', 'min', 'max']
	}).reset_index()

	fig, ax = plt.subplots(figsize=(10, 6))
	ax.plot(df['model_version'], df['opt']['mean'], marker='o', label='Optimized Mean')
	ax.fill_between(df['model_version'], df['opt']['min'], df['opt']['max'], alpha=0.2, label='Optimized Min-Max Range')
	ax.plot(df['model_version'], df['init']['mean'], marker='x', label='Initial Mean', linestyle=':')
	ax.set_xlabel('Iteration')
	ax.set_ylabel('Values')
	ax.set_title('Mean Objective Values of the Top Five Samples per Iteration')
	ax.set_ylim(0, 5)
	ax.axhline(y=2, color='gray', linestyle='--', label='Input Max')
	ax.legend()
	plt.xticks(ticks=range(len(df['model_version'])), labels=df['model_version'], rotation=45)

	plt.tight_layout()
	plt.show()

In [ ]:
scores_initial_optimized(version="ctrloralter_gbo_28")

### Original & Initial & Optimized

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def scores_original_initial_optimized(version):

	# Load results from the npz file
	results = np.load(f"../results/{version}/results.npz", allow_pickle=True)

	# Convert the npz file to a pandas DataFrame
	df = pd.DataFrame({
		'opt': list(results['opt_point_properties']),
		'init': list(results['init_point_properties']),
		'orig': list(results['orig_point_properties']),
		'model_version': list(results['opt_model_version']),
	})

	# Group by model version and aggregate
	df = df.groupby('model_version').agg({
		'opt': ['mean', 'min', 'max'],
		'init': ['mean', 'min', 'max'],
		'orig': ['mean', 'min', 'max']
	}).reset_index()

	fig, ax = plt.subplots(figsize=(10, 6))
	ax.plot(df['model_version'], df['opt']['mean'], marker='o', label='Optimized Mean')
	ax.fill_between(df['model_version'], df['opt']['min'], df['opt']['max'], alpha=0.2, label='Optimized Min-Max Range')
	ax.plot(df['model_version'], df['orig']['mean'], marker='s', label='Original Mean', linestyle='--')
	ax.plot(df['model_version'], df['init']['mean'], marker='x', label='Initial Mean', linestyle=':')
	ax.set_xlabel('Iteration')
	ax.set_ylabel('Values')
	ax.set_title('Mean Objective Values of the Top Five Samples per Iteration')
	ax.set_ylim(0, 5)
	ax.axhline(y=2, color='gray', linestyle='--', label='Input Max')
	ax.legend()
	plt.xticks(ticks=range(len(df['model_version'])), labels=df['model_version'], rotation=45)

	plt.tight_layout()
	plt.show()

In [ ]:
scores_original_initial_optimized("ctrloralter_gbo_23")

## Scores Compare

Compare optimized curves between multiple versions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


def scores_compare_optimized_mult_versions(version_dict):

	result_dict = {}
	min_iterations = np.inf

	for version, version_name in version_dict.items():
		# Load results from the npz file
		results = np.load(f"../results/{version}/results.npz", allow_pickle=True)

		# Convert the npz file to a pandas DataFrame
		df = pd.DataFrame({
			'opt': list(results['opt_point_properties']),
			'model_version': list(results['opt_model_version']),
		})

		# Group by model version and aggregate
		df = df.groupby('model_version').agg({'opt': 'mean'}).reset_index()

		# Load version
		version_name = version_name or f"Version {version}"

		# Store the results in the result_dict
		result_dict[version_name] = df['opt']

		# Update min_iterations
		min_iterations = min(min_iterations, len(df['model_version']))

	fig, ax = plt.subplots(figsize=(10, 6))

	for version_name, opt_values in result_dict.items():
		ax.plot(range(min_iterations), opt_values[:min_iterations], marker='o', label=f'{version_name}')

	ax.set_xlabel('Iteration')
	ax.set_ylabel('Values')
	ax.set_title('Mean Objective Values of Top Five Samples per Iteration')
	ax.set_ylim(0, 5)
	ax.axhline(y=2, color='gray', linestyle='--', label='Input Max')
	ax.legend()
	plt.xticks(ticks=range(min_iterations), labels=df['model_version'][:min_iterations], rotation=45)

	plt.tight_layout()
	plt.show()

In [ ]:
scores_compare_optimized_mult_versions({
    "ctrloralter_gbo_20": "LoRA Style",
    "ctrloralter_gbo_21": "LORA Style + Depth",
    "sd_gbo_03": "SD VAE",
    "latent_vqvae_gbo_06": "LatentVQVAE",
})

# Experiments

## Setup

### Results Directory

In [ ]:
from pathlib import Path

BASE_DIR = Path("../results").expanduser().resolve()

def get_result_dir(version : str, seed : int) -> Path:
    """
    Return the path to the first main.log that matches the seed.
    Allowed directory names:
      <version>_<seed>
      <version>_<seed>_<anything>
    """
    # exact match first (no job-id)
    exact = BASE_DIR / f"{version}_{seed}"
    if exact.is_dir():
        return exact

    # wildcard for any trailing underscore / job-id
    pattern = f"{version}_{seed}_*/"
    matches = sorted(BASE_DIR.glob(pattern))
    if not matches:
        raise FileNotFoundError(f"No log found for pattern {pattern} under {BASE_DIR}")
    return matches[-1]

## Experiment 1 (SD-VAE)

### SD1.5 Train Data vs. Normal Distribution

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image

import numpy as np
np.random.seed(42)  # For reproducibility

num_outer_rows = 8
num_outer_cols = 5
pair_cols = [0, 1, 3, 4]		# columns that get 2-image blocks (0,1 on the left of the spacer; 3,4 on the right)
data_rows = [0, 1, 3, 4, 6, 7]	# real data rows (skip the spacer rows 2 and 5)

fig = plt.figure(figsize=(8*2+0.2, 6*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 1, 0.01, 1, 1],
    height_ratios=[1, 1, 0.1, 1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
# rows that belong together: (0,1), (3,4), (6,7)
for upper_idx in range(0, len(data_rows), 2):
    top, bottom = data_rows[upper_idx], data_rows[upper_idx + 1]

    # iterate over the four 2-image blocks in the desired order
    for block_group in ([0, 1],          # left half  = training samples
                        [3, 4]):         # right half = normal-dist samples
        for pc in block_group:
            # inner grids for this block
            inner_top = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[top,    pc], wspace=0, hspace=0)
            inner_bottom = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[bottom, pc], wspace=0, hspace=0)

            # Append top inner grid
            axes.append(fig.add_subplot(inner_top[0, 0]))    # init
            axes.append(fig.add_subplot(inner_top[0, 1]))    # opt

            # Append bottom inner grid
            axes.append(fig.add_subplot(inner_bottom[0, 0])) # init
            axes.append(fig.add_subplot(inner_bottom[0, 1])) # opt

# Sample 4 random images from the results and convert to iter and img number
random_samples = np.random.choice(range(100), size=4, replace=False)
iter_nums = [x - (x % 5) for x in random_samples]
sample_nums = [x % 5 for x in random_samples]

# Get the images
samples = []
scores = []
for cell_idx, cell_name in enumerate(["dngo_train_lbfgsb", "dngo_normal_lbfgsb", "dngo_train_trustconstr", "dngo_normal_trustconstr", "gbo_train", "gbo_normal"]):

    # Define data directory
    result_dir = get_result_dir(f"ex1_sd15_{cell_name}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    for iter_num, sample_num in zip(iter_nums, sample_nums):

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(48):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.258, 1.04, "Training Data Samples", ha="center", va="center", fontsize=20)
fig.text(0.755, 1.04, "Normal Distribution Samples", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.02, 0.84, "L-BFGS-B", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.5, "trust-constr", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.16, "GBO", ha="center", va="center", fontsize=20, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal lines
fig.add_artist(plt.Line2D([0, 1], [0.68, 0.68], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.33, 0.33], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex1_sd15_train_normal.pdf", bbox_inches='tight')
plt.show()

### SD3.5 Original vs. Finetuned

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image

import numpy as np
np.random.seed(42)

num_outer_rows = 5
num_outer_cols = 5
pair_cols = [0, 1, 3, 4]
data_rows = [0, 1, 3, 4]

fig = plt.figure(figsize=(8*2+0.2, 4*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 1, 0.01, 1, 1],
    height_ratios=[1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for upper_idx in range(0, len(data_rows), 2):
    top, bottom = data_rows[upper_idx], data_rows[upper_idx + 1]

    for block_group in ([0, 1],
                        [3, 4]):
        for pc in block_group:
            # inner grids for this block
            inner_top = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[top,    pc], wspace=0.02, hspace=0)
            inner_bottom = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[bottom, pc], wspace=0.02, hspace=0)

            # Append top inner grid
            axes.append(fig.add_subplot(inner_top[0, 0]))    # init
            axes.append(fig.add_subplot(inner_top[0, 1]))    # opt

            # Append bottom inner grid
            axes.append(fig.add_subplot(inner_bottom[0, 0])) # init
            axes.append(fig.add_subplot(inner_bottom[0, 1])) # opt

# Sample 4 random images from the results and convert to iter and img number
random_samples = np.random.choice(range(100), size=4, replace=False)
iter_nums = [x - (x % 5) for x in random_samples]
sample_nums = [x % 5 for x in random_samples]

# Get the images
samples = []
scores = []
for cell_idx, cell_name in enumerate(["sd35_dngo_train_lbfgsb", "sd35f_dngo_train_lbfgsb", "sd35_dngo_train_trustconstr", "sd35f_dngo_train_trustconstr"]):

    # Define data directory
    result_dir = get_result_dir(f"ex1_{cell_name}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    for iter_num, sample_num in zip(iter_nums, sample_nums):

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(32):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.248, 1.06, "Original SD3.5", ha="center", va="center", fontsize=20)
fig.text(0.758, 1.06, "Finetuned SD3.5", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.02, 0.77, "L-BFGS-B", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.25, "trust-constr", ha="center", va="center", fontsize=20, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.51, 0.51], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex1_sd35_original_finetuned.pdf", bbox_inches='tight')
plt.show()

### SD3.5, PCA, FI

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 10
num_outer_cols = 5
img_rows = [0, 1, 3, 4, 6, 7, 9]
img_cols = [0, 2, 4]

fig = plt.figure(figsize=(6*2+0.2, 7*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1, 0.01, 1],
    height_ratios=[1, 1, 0.1, 1, 1, 0.1, 1, 1, 0.1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 2-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # init
        axes.append(fig.add_subplot(inner[0, 1]))  # opt

# Get the images
samples = []
scores = []
for cell_idx, cell_name in enumerate([
    "gp_train_lbfgsb", "gp_train_lbfgsb_pca", "gp_train_lbfgsb_fi",
    "dngo_train_lbfgsb", "dngo_train_lbfgsb_pca", "dngo_train_lbfgsb_fi",
    "gp_train_trustconstr", "gp_train_trustconstr_pca", "gp_train_trustconstr_fi",
    "dngo_train_trustconstr", "dngo_train_trustconstr_pca", "dngo_train_trustconstr_fi",
    "gp_train_trustconstrgmm", "gp_train_trustconstrgmm_pca", "gp_train_trustconstrgmm_fi",
    "dngo_train_trustconstrgmm", "dngo_train_trustconstrgmm_pca", "dngo_train_trustconstrgmm_fi",
    "gbo_train", "gbo_train_pca", "gbo_train_fi",
    ]):

    if cell_name in ["gp_train_trustconstrgmm", "dngo_train_trustconstrgmm"]:
        # # add white background
        samples.append(Image.new("RGB", (256, 256), (255, 255, 255)))
        samples.append(Image.new("RGB", (256, 256), (255, 255, 255)))
        scores.append("")
        scores.append("")
        continue

    # Define data directory
    result_dir = get_result_dir(f"ex1_sd35_{cell_name}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    # Get Top10 sample id
    top_10_idx = np.argsort(scores_file['opt_point_properties'])[-10]

    # Derive sample number
    iter_num = top_10_idx - (top_10_idx % 5)
    sample_num = top_10_idx % 5

    # Load initial image
    init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
    init_img = Image.open(init_file).convert("RGB")
    samples.append(init_img)
    scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

    # Load optimized image
    opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
    opt_img = Image.open(opt_file).convert("RGB")
    samples.append(opt_img)
    scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(42):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.15, 1.04, "SD3.5", ha="center", va="center", fontsize=20)
fig.text(0.5, 1.04, "SD3.5 + PCA", ha="center", va="center", fontsize=20)
fig.text(0.85, 1.04, "SD3.5 + FI", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.05, 0.88, "L-BFGS-B", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.945, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.813, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.582, "trust-constr", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.648, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.518, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.296, "trust-constr + GMM", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.358, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.225, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.04, 0.062, "GBO", ha="center", va="center", fontsize=20, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.325, 0.325], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0.672, 0.672], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.145, 0.145], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.44, 0.44], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.735, 0.735], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Plot a skull at certain coordinates
skull = Image.open("vis/fontawesome_skull.png").convert("RGBA")
# white background
skull_img = Image.new("RGB", skull.size, (255, 255, 255))
skull_img.paste(skull, (0, 0), skull)
# resize image
skull_img = skull_img.resize((int(skull_img.width * 0.3), int(skull_img.height * 0.3)))
fig.figimage(skull_img, 180, 350, alpha=1, zorder=1, origin='upper')

# plt.savefig("vis/ex1_sd35_pca_fi.pdf", bbox_inches='tight')
plt.show()

### Original vs. Initial samples

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 4
num_outer_cols = 3
img_rows = [0, 1, 2, 3]
img_cols = [0, 1, 2]

fig = plt.figure(figsize=(6*2+0.2, 4*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 2-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # init
        axes.append(fig.add_subplot(inner[0, 1]))  # opt

# Get the images
samples = []
scores = []
num_total_samples = 12
num_curr_samples = 0
for cell_idx, cell_name in enumerate([
    "gp_train_lbfgsb", "gp_train_lbfgsb_pca", "gp_train_lbfgsb_fi",
    "dngo_train_lbfgsb", "dngo_train_lbfgsb_pca", "dngo_train_lbfgsb_fi",
    "gp_train_trustconstr", "gp_train_trustconstr_pca", "gp_train_trustconstr_fi",
    "dngo_train_trustconstr", "dngo_train_trustconstr_pca", "dngo_train_trustconstr_fi",
    "gp_train_trustconstrgmm_pca", "gp_train_trustconstrgmm_fi",
    "dngo_train_trustconstrgmm_pca", "dngo_train_trustconstrgmm_fi",
    "gbo_train", "gbo_train_pca", "gbo_train_fi",
    ]):

    # Define data directory
    result_dir = get_result_dir(f"ex1_sd35_{cell_name}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    # Reduce scores file to the first iteration
    scores_file = {k: v[:5] for k, v in scores_file.items() if k in ['init_point_properties', 'orig_point_properties']}

    # Find indices where initial score is larger than original score
    larger_than_original = np.where(scores_file['init_point_properties'] > scores_file['orig_point_properties'])[0]

    for idx in larger_than_original:

        # Derive sample number
        iter_num = idx - (idx % 5)
        sample_num = idx % 5

        # Load original image
        orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
        orig_img = Image.open(orig_file).convert("RGB")
        samples.append(orig_img)
        scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

# Remove duplicate images if we have more than 12 samples
for i in range(0, len(samples), 2):
    for j in range(i + 2, len(samples), 2):
        if samples[i] == samples[j]:
            # If we found a duplicate, we can remove it
            samples.pop(j)
            samples.pop(j)
            scores.pop(j)
            scores.pop(j)
            break

# Plot images on the axes
for i in range(24):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

plt.suptitle("Samples with Initial Smile Score > Original Smile Score", fontsize=20, y=1.08)

# plt.savefig("vis/ex1_sd35_original_initial.pdf", bbox_inches='tight')
plt.show()

## Experiment 3 (LatentVQVAE)

In [ ]:
# Check if all runs completed all iterations
for cell in [
    "gp_train_lbfgsb", "dngo_train_lbfgsb",
    "gp_train_trustconstr", "dngo_train_trustconstr",
    "gp_train_trustconstrgmm", "dngo_train_trustconstrgmm",
	"gp_normal_lbfgsb", "dngo_normal_lbfgsb",
	"gp_normal_trustconstr", "dngo_normal_trustconstr",
	"gp_normal_trustconstrgmm", "dngo_normal_trustconstrgmm",
	"gp_train_lbfgsb_fi", "dngo_train_lbfgsb_fi",
	"gp_train_trustconstr_fi", "dngo_train_trustconstr_fi",
	"gp_train_trustconstrgmm_fi", "dngo_train_trustconstrgmm_fi",
	"gbo_train", "gbo_normal", "gbo_train_fi",
    ]:
	for seed in [42, 43, 44]:
		try:
			result_dir = get_result_dir(f"ex3_{cell}", seed=seed)
			data_dir = result_dir / "data/samples"
			scores_file = np.load(result_dir / "results.npz")
			print(f"{cell} (seed {seed}): {len(scores_file['opt_point_properties'])}")
		except FileNotFoundError:
			print(f"{cell} (seed {seed}): No results found")

### SparseGP vs. DNGO, LBFGSB vs. TrustConstr

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 8
num_outer_cols = 3
img_rows = [0, 1, 3, 4, 6, 7]
img_cols = [0, 2]

fig = plt.figure(figsize=(6*2+0.2, 6*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1],
    height_ratios=[1, 1, 0.1, 1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 3-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # orig
        axes.append(fig.add_subplot(inner[0, 1]))  # init
        axes.append(fig.add_subplot(inner[0, 2]))  # opt

# Get the images
samples = []
scores = []
for cell_idx, cell_name in enumerate([
    "gp_train_lbfgsb", "dngo_train_lbfgsb",
    "gp_train_trustconstr", "dngo_train_trustconstr",
    "gp_train_trustconstrgmm", "dngo_train_trustconstrgmm",
    ]):

    # Define data directory
    result_dir = get_result_dir(f"ex3_{cell_name}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    # Get Top10 and Top50 sample id
    top_10_idx = np.argsort(scores_file['opt_point_properties'])[-10]
    top_50_idx = np.argsort(scores_file['opt_point_properties'])[-50]

    for top_idx in [top_10_idx, top_50_idx]:

        # Derive sample number
        iter_num = top_idx - (top_idx % 5)
        sample_num = top_idx % 5

        # Load original image
        orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
        orig_img = Image.open(orig_file).convert("RGB")
        samples.append(orig_img)
        scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(36):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.235, 1.04, "Top10", ha="center", va="center", fontsize=20)
fig.text(0.765, 1.04, "Top50", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.05, 0.86, "L-BFGS-B", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.935, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.78, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.51, "trust-constr", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.58, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.425, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.152, "trust-constr + GMM", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.225, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.07, "DNGO", ha="center", va="center", fontsize=14, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.685, 0.685], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex3_bo_comparison.pdf", bbox_inches='tight')
plt.show()

### Iterations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 8
num_outer_cols = 1
img_rows = [0, 1, 3, 4, 6, 7]
img_cols = [0]

fig = plt.figure(figsize=(5*2+0.2, 6*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1],
    height_ratios=[1, 1, 0.1, 1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 5-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 5, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        for i in range(5):
            axes.append(fig.add_subplot(inner[0, i]))

# Get the images
samples = []
scores = []

for iter_num in [0, 5, 10]: # first 3 iterations only

    for cell_idx, cell_name in enumerate(["gp_train_lbfgsb", "dngo_train_lbfgsb"]):

        # Define data directory
        result_dir = get_result_dir(f"ex3_{cell_name}", seed=42)
        data_dir = result_dir / "data/samples"
        scores_file = np.load(result_dir / "results.npz")

        # Load all images for the current iteration
        for sample_num in range(5):

            # Load optimized image
            opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
            opt_img = Image.open(opt_file).convert("RGB")
            samples.append(opt_img)
            scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(30):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Row titles
fig.text(-0.05, 0.86, "Iteration 0", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.935, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.78, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.51, "Iteration 1", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.58, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.425, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.152, "Iteration 2", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.225, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.07, "DNGO", ha="center", va="center", fontsize=14, rotation=90)

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.685, 0.685], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex3_bo_iteration.pdf", bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 11
num_outer_cols = 1
img_rows = [0, 1, 2, 4, 5, 6, 8, 9, 10]
img_cols = [0]

fig = plt.figure(figsize=(5*2+0.2, 9*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1],
    height_ratios=[1, 1, 1, 0.1, 1, 1, 1, 0.1, 1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 5-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 5, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        for i in range(5):
            axes.append(fig.add_subplot(inner[0, i]))

# Get the images
samples = []
scores = []

for iter_num in [0, 5, 10]: # first 3 iterations only

    for cell_idx, cell_name in enumerate(["gp_train_lbfgsb", "dngo_train_lbfgsb", "gbo_train"]):

        # Define data directory
        result_dir = get_result_dir(f"ex3_{cell_name}", seed=42)
        data_dir = result_dir / "data/samples"
        scores_file = np.load(result_dir / "results.npz")

        # Load all images for the current iteration
        for sample_num in range(5):

            # Load optimized image
            opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
            opt_img = Image.open(opt_file).convert("RGB")
            samples.append(opt_img)
            scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(45):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Row titles
fig.text(-0.05, 0.85, "Iteration 0", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.96, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.85, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.74, "GBO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.50, "Iteration 1", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.61, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.50, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.395, "GBO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.152, "Iteration 2", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.26, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.155, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.045, "GBO", ha="center", va="center", fontsize=14, rotation=90)

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.68, 0.68], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex3_iteration.pdf", bbox_inches='tight')
plt.show()

### Sampling Distribution

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 5
num_outer_cols = 3
img_rows = [0, 1, 3, 4]
img_cols = [0, 2]

fig = plt.figure(figsize=(5*2+0.2, 4*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 0.67],
    height_ratios=[1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    # Left column: 3 images
    inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, 0], wspace=0.02, hspace=0)
    axes.append(fig.add_subplot(inner[0, 0]))  # orig
    axes.append(fig.add_subplot(inner[0, 1]))  # init
    axes.append(fig.add_subplot(inner[0, 2]))  # opt

    # Right column: 2 images
    inner = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[row_idx, 2], wspace=0.02, hspace=0)
    axes.append(fig.add_subplot(inner[0, 0]))  # init
    axes.append(fig.add_subplot(inner[0, 1]))  # opt

samples = []
scores = []

# BO
for top_x in [10, 50]:
    # 1) Training data samples
    result_dir = get_result_dir(f"ex3_dngo_train_lbfgsb", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    # Get index
    top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

    # Derive sample number
    iter_num = top_idx - (top_idx % 5)
    sample_num = top_idx % 5

    # Load original image
    orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
    orig_img = Image.open(orig_file).convert("RGB")
    samples.append(orig_img)
    scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

    # Load initial image
    init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
    init_img = Image.open(init_file).convert("RGB")
    samples.append(init_img)
    scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

    # Load optimized image
    opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
    opt_img = Image.open(opt_file).convert("RGB")
    samples.append(opt_img)
    scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

    # 2) Normal distribution samples
    result_dir = get_result_dir(f"ex3_dngo_normal_lbfgsb", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    # Get index
    top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

    # Derive sample number
    iter_num = top_idx - (top_idx % 5)
    sample_num = top_idx % 5

    # Load initial image
    init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
    init_img = Image.open(init_file).convert("RGB")
    samples.append(init_img)
    scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

    # Load optimized image
    opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
    opt_img = Image.open(opt_file).convert("RGB")
    samples.append(opt_img)
    scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# GBO
for top_x in [10, 50]:
    for sampling_dist in ["train", "normal"]:
        # Define data directory
        result_dir = get_result_dir(f"ex3_gbo_{sampling_dist}", seed=42)
        data_dir = result_dir / "data/samples"
        scores_file = np.load(result_dir / "results.npz")

        # Get index
        top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

        # Derive sample number
        iter_num = top_idx - (top_idx % 5)
        sample_num = top_idx % 5

        # Load original image (only for train)
        if sampling_dist == "train":
            orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
            orig_img = Image.open(orig_file).convert("RGB")
            samples.append(orig_img)
            scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(20):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.28, 1.06, "Training Data Samples", ha="center", va="center", fontsize=20)
fig.text(0.82, 1.06, "Normal Dist. Samples", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.05, 0.78, "BO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.9, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.65, "Top50", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.24, "GBO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.35, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.11, "Top50", ha="center", va="center", fontsize=14, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.592, 0.592], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.51, 0.51], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex3_bo_gbo_train_normal.pdf", bbox_inches='tight')
plt.show()

### Feature Selection

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 5
num_outer_cols = 3
img_rows = [0, 1, 3, 4]
img_cols = [0, 2]

fig = plt.figure(figsize=(6*2+0.2, 4*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1],
    height_ratios=[1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 3-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # orig
        axes.append(fig.add_subplot(inner[0, 1]))  # init
        axes.append(fig.add_subplot(inner[0, 2]))  # opt

samples = []
scores = []
for cell_name in ["dngo_train_lbfgsb", "gbo_train"]:
	for top_x in [10, 50]:
		for fs in ["", "_fi"]:

			result_dir = get_result_dir(f"ex3_{cell_name}{fs}", seed=42)
			data_dir = result_dir / "data/samples"
			scores_file = np.load(result_dir / "results.npz")

			# Get index
			top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

			# Derive sample number
			iter_num = top_idx - (top_idx % 5)
			sample_num = top_idx % 5

			# Load original image
			orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
			orig_img = Image.open(orig_file).convert("RGB")
			samples.append(orig_img)
			scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

			# Load initial image
			init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
			init_img = Image.open(init_file).convert("RGB")
			samples.append(init_img)
			scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

			# Load optimized image
			opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
			opt_img = Image.open(opt_file).convert("RGB")
			samples.append(opt_img)
			scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(24):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.235, 1.06, "LatentVQVAE", ha="center", va="center", fontsize=20)
fig.text(0.765, 1.06, "LatentVQVAE + FI", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.05, 0.78, "BO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.9, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.65, "Top50", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.24, "GBO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.35, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.11, "Top50", ha="center", va="center", fontsize=14, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.51, 0.51], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex3_feature_selection.pdf", bbox_inches='tight')
plt.show()

## Experiment 4 (LoRAdapter)

In [ ]:
# Check if all runs completed all iterations
for cell in [
	"sd15_gp_style", "sd15_gp_styledepth",
    "sd15_dngo_style", "sd15_dngo_styledepth",
	"sd15_gbo_style", "sd15_gbo_styledepth",
	"sdxl_gp", "sdxl_dngo", "sdxl_gbo"
    ]:
	for seed in [42, 43, 44]:
		try:
			result_dir = get_result_dir(f"ex4_{cell}", seed=seed)
			data_dir = result_dir / "data/samples"
			scores_file = np.load(result_dir / "results.npz")
			print(f"{cell} (seed {seed}): {len(scores_file['opt_point_properties'])}")
		except FileNotFoundError:
			print(f"{cell} (seed {seed}): No results found")

### SD1.5

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 8
num_outer_cols = 3
img_rows = [0, 1, 3, 4, 6, 7]
img_cols = [0, 2]

fig = plt.figure(figsize=(6*2+0.2, 6*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1],
    height_ratios=[1, 1, 0.1, 1, 1, 0.1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 3-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # orig
        axes.append(fig.add_subplot(inner[0, 1]))  # init
        axes.append(fig.add_subplot(inner[0, 2]))  # opt

# Get the images
samples = []
scores = []
for method in ["gp", "dngo", "gbo"]:
    for top_x in [10, 50]:
        for cell_name in [f"sd15_{method}_style", f"sd15_{method}_styledepth"]:

            # Define data directory
            result_dir = get_result_dir(f"ex4_{cell_name}", seed=42)
            data_dir = result_dir / "data/samples"
            scores_file = np.load(result_dir / "results.npz")

            # Get TopK sample id
            top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

            # Derive sample number
            iter_num = top_idx - (top_idx % 5)
            sample_num = top_idx % 5

            # Load original image
            orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
            orig_img = Image.open(orig_file).convert("RGB")
            samples.append(orig_img)
            scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

            # Load initial image
            init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
            init_img = Image.open(init_file).convert("RGB")
            samples.append(init_img)
            scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

            # Load optimized image
            opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
            opt_img = Image.open(opt_file).convert("RGB")
            samples.append(opt_img)
            scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(36):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.24, 1.04, "SD1.5", ha="center", va="center", fontsize=20)
fig.text(0.775, 1.04, "SD1.5 + Depth cond.", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.05, 0.86, "SparseGP", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.935, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.78, "Top50", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.51, "DNGO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.58, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.425, "Top50", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.152, "GBO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.225, "Top10", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.07, "Top50", ha="center", va="center", fontsize=14, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.685, 0.685], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

plt.savefig("vis/ex4_sd15.pdf", bbox_inches='tight')
plt.show()

### SDXL

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 5
num_outer_cols = 3
img_rows = [0, 2, 4]
img_cols = [0, 2]

fig = plt.figure(figsize=(6*2+0.1, 3*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1],
    height_ratios=[1, 0.1, 1, 0.1, 1],
    wspace=0.05,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 3-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # orig
        axes.append(fig.add_subplot(inner[0, 1]))  # init
        axes.append(fig.add_subplot(inner[0, 2]))  # opt

# Get the images
samples = []
scores = []
for method in ["gp", "dngo", "gbo"]:

    # Define data directory
    result_dir = get_result_dir(f"ex4_sdxl_{method}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    for top_x in [10, 50]:

        # Get TopK sample id
        top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

        # Derive sample number
        iter_num = top_idx - (top_idx % 5)
        sample_num = top_idx % 5

        # Load original image
        orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
        orig_img = Image.open(orig_file).convert("RGB")
        samples.append(orig_img)
        scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(18):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.242, 1.07, "Top10", ha="center", va="center", fontsize=20)
fig.text(0.762, 1.07, "Top50", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.02, 0.868, "SparseGP", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.51, "DNGO", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.14, "GBO", ha="center", va="center", fontsize=20, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.69, 0.69], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex4_sdxl.pdf", bbox_inches='tight')
plt.show()

### Iterations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 11
num_outer_cols = 1
img_rows = [0, 1, 2, 4, 5, 6, 8, 9, 10]
img_cols = [0]

fig = plt.figure(figsize=(5*2+0.2, 9*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1],
    height_ratios=[1, 1, 1, 0.1, 1, 1, 1, 0.1, 1, 1, 1],
    wspace=0.1,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 5-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 5, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        for i in range(5):
            axes.append(fig.add_subplot(inner[0, i]))

# Get the images
samples = []
scores = []

for iter_num in [0, 5, 10]: # first 3 iterations only

    for cell_idx, cell_name in enumerate(["gp", "dngo", "gbo"]):

        # Define data directory
        result_dir = get_result_dir(f"ex4_sd15_{cell_name}_style", seed=42)
        data_dir = result_dir / "data/samples"
        scores_file = np.load(result_dir / "results.npz")

        # Load all images for the current iteration
        for sample_num in range(5):

            # Load optimized image
            opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
            opt_img = Image.open(opt_file).convert("RGB")
            samples.append(opt_img)
            scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(45):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Row titles
fig.text(-0.05, 0.85, "Iteration 0", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.96, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.85, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.74, "GBO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.50, "Iteration 1", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.61, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.50, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.395, "GBO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.05, 0.152, "Iteration 2", ha="center", va="center", fontsize=20, rotation=90)
fig.text(-0.02, 0.26, "SparseGP", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.155, "DNGO", ha="center", va="center", fontsize=14, rotation=90)
fig.text(-0.02, 0.045, "GBO", ha="center", va="center", fontsize=14, rotation=90)

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.333, 0.333], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0, 1], [0.68, 0.68], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex4_iteration.pdf", bbox_inches='tight')
plt.show()

### Retraining

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 3
num_outer_cols = 3
img_rows = [0, 2]
img_cols = [0, 2]

fig = plt.figure(figsize=(6*2+0.1, 2*2*1.05+0.2))

# Outer grid
outer = gridspec.GridSpec(
    nrows=num_outer_rows,
    ncols=num_outer_cols,
    figure=fig,
    width_ratios=[1, 0.01, 1],
    height_ratios=[1, 0.1, 1],
    wspace=0.05,
    hspace=0.2,
    bottom=0, top=1,
    left=0, right=1,
)

axes = []
for row_idx in img_rows:
    for col_idx in img_cols:
        # Create inner gridspec for each 3-image block
        inner = gridspec.GridSpecFromSubplotSpec(1, 3, subplot_spec=outer[row_idx, col_idx], wspace=0.02, hspace=0)
        # Append axes for the two images in the block
        axes.append(fig.add_subplot(inner[0, 0]))  # orig
        axes.append(fig.add_subplot(inner[0, 1]))  # init
        axes.append(fig.add_subplot(inner[0, 2]))  # opt

# Get the images
samples = []
scores = []
for method in ["long", "long_retrain"]:

    # Define data directory
    result_dir = get_result_dir(f"ex4_sd15_dngo_{method}", seed=42)
    data_dir = result_dir / "data/samples"
    scores_file = np.load(result_dir / "results.npz")

    for top_x in [10, 50]:

        # Get TopK sample id
        top_idx = np.argsort(scores_file['opt_point_properties'])[-top_x]

        # Derive sample number
        iter_num = top_idx - (top_idx % 5)
        sample_num = top_idx % 5

        # Load original image
        orig_file = data_dir / f"iter_{iter_num}/img_orig/{sample_num}.png"
        orig_img = Image.open(orig_file).convert("RGB")
        samples.append(orig_img)
        scores.append(f"Original: {scores_file['orig_point_properties'][iter_num + sample_num].item():.2f}")

        # Load initial image
        init_file = data_dir / f"iter_{iter_num}/img_init/{sample_num}.png"
        init_img = Image.open(init_file).convert("RGB")
        samples.append(init_img)
        scores.append(f"Initial: {scores_file['init_point_properties'][iter_num + sample_num].item():.2f}")

        # Load optimized image
        opt_file = data_dir / f"iter_{iter_num}/img_opt/{sample_num}.png"
        opt_img = Image.open(opt_file).convert("RGB")
        samples.append(opt_img)
        scores.append(f"Optimized: {scores_file['opt_point_properties'][iter_num + sample_num].item():.2f}")

# Plot images on the axes
for i in range(12):
    ax = axes[i]
    ax.imshow(samples[i])
    ax.set_title(f"{scores[i]}")
    # Remove ticks and spines
    ax.set_xticks([])
    ax.set_yticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

# Column titles
fig.text(0.242, 1.1, "Top10", ha="center", va="center", fontsize=20)
fig.text(0.762, 1.1, "Top50", ha="center", va="center", fontsize=20)

# Row titles
fig.text(-0.02, 0.79, "w/o Retraining", ha="center", va="center", fontsize=18, rotation=90)
fig.text(-0.02, 0.21, "w/ Retraining", ha="center", va="center", fontsize=18, rotation=90)

# Vertical line
fig.add_artist(plt.Line2D([0.5, 0.5], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# Horizontal line
fig.add_artist(plt.Line2D([0, 1], [0.52, 0.52], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex4_retraining.pdf", bbox_inches='tight')
plt.show()

## Experiment 5 (Comparison)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
import numpy as np

num_outer_rows = 5
num_outer_cols = 10
img_rows = [0, 1, 2, 3, 4]
img_cols = [0, 2, 3, 5, 6, 8, 9]

fig = plt.figure(figsize=(7*2+0.1, 5*2+0.2))

# Outer grid
outer = gridspec.GridSpec(
	nrows=num_outer_rows,
	ncols=num_outer_cols,
	figure=fig,
	width_ratios=[1, 0.03, 1, 1, 0.03, 1, 1, 0.03, 1, 1],
	height_ratios=[1, 1, 1, 1, 1],
	wspace=0.05,
	hspace=0.1,
	bottom=0, top=1,
	left=0, right=1,
)

axes = []
for row_idx in img_rows:
	for col_idx in img_cols:
		axes.append(fig.add_subplot(outer[row_idx, col_idx]))

# Images
iter_images = {
	# Iteration: SD-VAE idx, LatentVQVAE idx, CTRLorALTer idx
	'iter_0': [0, 0, 0],
	# 'iter_5': [0, 0, 0],
	'iter_10': [3, 1, 0],
	# 'iter_15': [1, 0, 1],
	'iter_20': [2, 2, 3],
	'iter_25': [4, 3, 3],
	# 'iter_30': [2, 4, 4],
	'iter_35': [1, 3, 3],
	# 'iter_40': [0, 1, 0],
	# 'iter_45': [1, 0, 0],
}

# Get the images
samples = []
for iter, (sd_idx, lv_idx, ctrl_idx) in iter_images.items():

	# Original + SD-VAE
	result_dir = get_result_dir(f"ex5_sd35_dngo_short", seed=42)
	data_dir = result_dir / f"data/samples/{iter}"

	# Load original image
	orig_file = data_dir / f"img_orig/{sd_idx}.png"
	orig_img = Image.open(orig_file).convert("RGB")
	samples.append(orig_img)

	# Load SD-VAE initial
	init_file = data_dir / f"img_init/{sd_idx}.png"
	init_img = Image.open(init_file).convert("RGB")
	samples.append(init_img)

	# Load SD-VAE optimized
	opt_file = data_dir / f"img_opt/{sd_idx}.png"
	opt_img = Image.open(opt_file).convert("RGB")
	samples.append(opt_img)

	# LatentVQVAE
	result_dir = get_result_dir(f"ex5_latentvqvae_dngo_short", seed=42)
	data_dir = result_dir / f"data/samples/{iter}"

	# Load LatentVQVAE initial
	init_file = data_dir / f"img_init/{lv_idx}.png"
	init_img = Image.open(init_file).convert("RGB")
	samples.append(init_img)

	# Load LatentVQVAE optimized
	opt_file = data_dir / f"img_opt/{lv_idx}.png"
	opt_img = Image.open(opt_file).convert("RGB")
	samples.append(opt_img)

	# CTRLorALTer
	result_dir = get_result_dir(f"ex5_ctrloralter_dngo_short", seed=42)
	data_dir = result_dir / f"data/samples/{iter}"

	# Load CTRLorALTer initial
	init_file = data_dir / f"img_init/{ctrl_idx}.png"
	init_img = Image.open(init_file).convert("RGB")
	samples.append(init_img)

	# Load CTRLorALTer optimized
	opt_file = data_dir / f"img_opt/{ctrl_idx}.png"
	opt_img = Image.open(opt_file).convert("RGB")
	samples.append(opt_img)

# Plot images on the axes
for i in range(35):
	ax = axes[i]
	ax.imshow(samples[i])
	# Remove ticks and spines
	ax.set_xticks([])
	ax.set_yticks([])
	for spine in ax.spines.values():
		spine.set_visible(False)

# # Column titles
fig.text(0.07, 1.015, "Original", ha="center", va="center", fontsize=18)
fig.text(0.284, 1.04, "LSO-SD", ha="center", va="center", fontsize=20, fontweight='bold')
fig.text(0.218, 1.015, "Initial", ha="center", va="center", fontsize=18)
fig.text(0.359, 1.015, "Optimized", ha="center", va="center", fontsize=18)
fig.text(0.582, 1.04, "LSO-LatentVQVAE", ha="center", va="center", fontsize=20, fontweight='bold')
fig.text(0.506, 1.015, "Initial", ha="center", va="center", fontsize=18)
fig.text(0.647, 1.015, "Optimized", ha="center", va="center", fontsize=18)
fig.text(0.86, 1.04, "LoRASO", ha="center", va="center", fontsize=20, fontweight='bold')
fig.text(0.794, 1.015, "Initial", ha="center", va="center", fontsize=18)
fig.text(0.935, 1.015, "Optimized", ha="center", va="center", fontsize=18)

# # Vertical line
fig.add_artist(plt.Line2D([0.1415, 0.1415], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0.43, 0.43], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))
fig.add_artist(plt.Line2D([0.718, 0.718], [0, 1], transform=fig.transFigure, color='gray', linewidth=2, linestyle=':'))

# plt.savefig("vis/ex5_comparison.pdf", bbox_inches='tight')
plt.show()